## Step 1: Data Preparation

In [ ]:
import os

print(f"Previous exec path: {os.getcwd()}")
# move two level up
os.chdir("../../")
print(f"Current exec path: {os.getcwd()}")

from source.utils.dot2polygon import dot2polygon
import glob
import yaml

Previous exec path: /workspace/source/notebooks
Current exec path: /workspace


In [2]:
# make a folder for converted annotations
output_root = r"./data/monkey-data/annotations_polygon"
if not (os.path.isdir(output_root)):
    os.mkdir(output_root)

Convert the dot annotation to polygon bounding boxes

In [3]:
# TODO: check those values because in the eval they changed the numbers!
lymphocyte_half_box_size = (
    4.5  # the size of half of the bbox around the lymphocyte dot in um
)
# NOTE: reduced this to 5.0 as the eval script (it was 11.0)
monocytes_half_box_size = (
    5.0  # the size of half of the bbox around the monocytes dot in um
)
min_spacing = 0.25  # NOTE: in the eval code they use 0.24199951445730394
# spacing is the zoom level of the image, in micro-meters per pixel (was rounded to 0.25)

In [5]:
# convert dot annotations to polygon, single image

# xml_path = r'./data/monkey-data/annotations/xml/A_P000001.xml'
# output_path = r'./data/monkey-data/annotations_polygon/A_P000001_polygon.xml'

# convert the dot annotations (in mm) to polygon bounding boxes
# dot2polygon(xml_path, lymphocyte_half_box_size, monocytes_half_box_size, min_spacing, output_path)

In [6]:
# convert dot annotations to polygon, in a folder

annotation_dir = r"./data/monkey-data/annotations/xml"
annotation_polygon_dir = r"./data/monkey-data/annotations_polygon"
annotation_list = glob.glob(os.path.join(annotation_dir, "*.xml"))

print(annotation_list)
for xml_path in annotation_list:
    print(xml_path)
    output_path = os.path.join(
        annotation_polygon_dir,
        os.path.splitext(os.path.basename(xml_path))[0]
        + "_polygon"
        + os.path.splitext(os.path.basename(xml_path))[1],
    )
    print(output_path)
    # lymphocyte_half_box_size = 4.5 # the size of half of the bbox around the lymphocyte dot in um
    # monocytes_half_box_size = 11.0 # the size of half of the bbox around the monocytes dot in um
    # min_spacing = 0.25
    dot2polygon(
        xml_path,
        lymphocyte_half_box_size,
        monocytes_half_box_size,
        min_spacing,
        output_path,
    )

['./data/monkey-data/annotations/xml/D_P000018.xml', './data/monkey-data/annotations/xml/B_P000019.xml', './data/monkey-data/annotations/xml/A_P000033.xml', './data/monkey-data/annotations/xml/D_P000012.xml', './data/monkey-data/annotations/xml/D_P000013.xml', './data/monkey-data/annotations/xml/A_P000017.xml', './data/monkey-data/annotations/xml/A_P000030.xml', './data/monkey-data/annotations/xml/C_P000022.xml', './data/monkey-data/annotations/xml/C_P000021.xml', './data/monkey-data/annotations/xml/C_P000036.xml', './data/monkey-data/annotations/xml/B_P000016.xml', './data/monkey-data/annotations/xml/D_P000011.xml', './data/monkey-data/annotations/xml/C_P000023.xml', './data/monkey-data/annotations/xml/A_P000004.xml', './data/monkey-data/annotations/xml/D_P000010.xml', './data/monkey-data/annotations/xml/B_P000014.xml', './data/monkey-data/annotations/xml/A_P000020.xml', './data/monkey-data/annotations/xml/B_P000002.xml', './data/monkey-data/annotations/xml/C_P000033.xml', './data/mon

Creating the yaml config files for training.

In [7]:
def folders_to_yml(wsi_dir: str, wsa_dir: str, output_dir: str, output_name: str):
    """
    Generate a yaml file to be used as WSD dataconfig from a folder of slides and a folder of annotation or mask files.
    Assumes files use the same name for both the slides and masks.
    """

    wsa_list = glob.glob(wsa_dir)

    yaml_dict = {"training": []}
    # yaml_dict = {'training': [], 'validation': []}
    for wsa in wsa_list:
        patient_name = os.path.basename(wsa).split(
            os.path.basename(wsa_dir).split("*")[1]
        )[0]  # monocytes
        #     print(patient_name)
        if os.path.isfile(os.path.join(wsi_dir, patient_name + "_PAS_CPG.tif")):
            wsi = os.path.join(wsi_dir, patient_name + "_PAS_CPG.tif")
            print("match found:    ", patient_name)
            yaml_dict["training"].append(
                {"wsa": {"path": str(wsa)}, "wsi": {"path": str(wsi)}}
            )

            # # validation if needed
            # yaml_dict['validation'].append(
            #         {"wsa": {"path": str(wsa)}, "wsi": {"path": str(wsi)}})

        else:
            print("no match found:    ", patient_name)

    # make a folder for output
    if not (os.path.isdir(output_dir)):
        os.mkdir(output_dir)

    with open(os.path.join(output_dir, output_name), "w") as file:
        yaml.safe_dump(yaml_dict, file)

In [8]:
wsi_dir = r"./data/monkey-data/images/pas-cpg"  # r'./data_debug/images/pas-cpg'
wsa_dir = r"./data/monkey-data/annotations_polygon/*_polygon.xml"
output_dir = r"./source/configs"
output_name = "training_full.yml"

folders_to_yml(wsi_dir, wsa_dir, output_dir, output_name)

match found:     B_P000004
match found:     D_P000011
match found:     B_P000005
match found:     A_P000036
match found:     A_P000001
match found:     D_P000004
match found:     A_P000011
match found:     C_P000031
match found:     A_P000029
match found:     D_P000010
match found:     C_P000033
match found:     A_P000035
match found:     D_P000015
match found:     C_P000036
match found:     A_P000006
match found:     A_P000018
match found:     B_P000003
match found:     B_P000016
match found:     A_P000034
match found:     C_P000037
match found:     D_P000002
match found:     B_P000007
match found:     B_P000015
match found:     D_P000001
match found:     D_P000009
match found:     A_P000004
match found:     C_P000023
match found:     A_P000003
match found:     A_P000005
match found:     A_P000014
match found:     C_P000035
match found:     C_P000024
match found:     D_P000016
match found:     C_P000038
match found:     B_P000020
match found:     D_P000007
match found:     C_P000039
m

## Data Preparation Using WSI Metadatas

In [2]:
from source.utils.config_parser import get_args_and_config

os.chdir("source")

args, config = get_args_and_config()

2024-12-19 11:28:16,905 - INFO - argparser - 
Trying to load configuration file from specified or default '--config' args...
2024-12-19 11:28:16,919 - INFO - argparser - Loaded configuration from ./configs/baseline/detectron2_baseline.yml:
2024-12-19 11:28:16,924 - INFO - argparser - Load of args and config completed!
Loaded the following args:
{   'config': './configs/baseline/detectron2_baseline.yml',
    'debug': False,
    'train': True,
    'use_wandb': True}
Loaded the following configs:
{   'dataset': {   'annotation_polygon_dir': 'annotations_polygon',
                   'lymphocyte_half_box_size': 4.5,
                   'min_spacing': 0.25,
                   'monocytes_half_box_size': 5.0,
                   'name': 'monkey_dataset',
                   'path': '../data/monkey-data',
                   'yaml_wsi_wsa_path': './configs/training_full.yml'},
    'model': {'name': 'detectron2'},
    'optimizer': None,
    'other': None,
    'project': {   'continue_training': Fals

In [13]:
name = config["model"].get("name", "unet")

'detectron2'

In [3]:
dataset_config = config["dataset"]
dataset_config

{'name': 'monkey_dataset',
 'path': '../data/monkey-data',
 'annotation_polygon_dir': 'annotations_polygon',
 'yaml_wsi_wsa_path': './configs/training_full.yml',
 'lymphocyte_half_box_size': 4.5,
 'monocytes_half_box_size': 5.0,
 'min_spacing': 0.25}

In [4]:
dataset_path = dataset_config["path"]
dataset_annotation_path = os.path.join(
    dataset_path, dataset_config["annotation_polygon_dir"]
)
metadata_file = os.path.join(dataset_path, "metadata", "context-information.xlsx")

In [6]:
import pandas as pd

!pip install openpyxl
metadata_df = pd.read_excel(metadata_file, index_col=0)
# remove index
metadata_df.reset_index(drop=True, inplace=True)
metadata_df.head(3)

,Slide ID,Lab,Diagnosis,CD3CD20_IHC_quality,PU.1_IHC_quality,IR_CPG-IHC,IR_CPG-Diagnostic,IR_CPG-Original,Nb_lymphocytes,Nb_monocytes,Nb_rois
0,A_P000001,Radboud,TCMR 2B,Good,"Acceptable, strong PU1 intenisty with also bac...",Good,x,x,485,1185,2
1,A_P000002,Radboud,Borderline rejection (Mild ptc and focally sus...,Good,"Acceptable, strong PU1 intenisty with also bac...",Good,x,x,1063,214,1
2,A_P000003,Radboud,TCMR IB (Severe ptc and tubulitis),Good,"Good, moderate PU1 intensity",Good,x,x,549,265,1


In [7]:
metadata_df.replace("x", "Not available", inplace=True)
metadata_df.head(3)

,Slide ID,Lab,Diagnosis,CD3CD20_IHC_quality,PU.1_IHC_quality,IR_CPG-IHC,IR_CPG-Diagnostic,IR_CPG-Original,Nb_lymphocytes,Nb_monocytes,Nb_rois
0,A_P000001,Radboud,TCMR 2B,Good,"Acceptable, strong PU1 intenisty with also bac...",Good,Not available,Not available,485,1185,2
1,A_P000002,Radboud,Borderline rejection (Mild ptc and focally sus...,Good,"Acceptable, strong PU1 intenisty with also bac...",Good,Not available,Not available,1063,214,1
2,A_P000003,Radboud,TCMR IB (Severe ptc and tubulitis),Good,"Good, moderate PU1 intensity",Good,Not available,Not available,549,265,1


In [8]:
metadata_df.iloc[0]["Slide ID"]

'A_P000001'

In [9]:
for row in metadata_df.iterrows():
    row = row[1]
    print(row["Slide ID"])

A_P000001
A_P000002
A_P000003
A_P000004
A_P000005
A_P000006
A_P000007
A_P000011
A_P000014
A_P000016
A_P000017
A_P000018
A_P000020
A_P000021
A_P000022
A_P000024
A_P000029
A_P000030
A_P000031
A_P000032
A_P000033
A_P000034
A_P000035
A_P000036
A_P000037
A_P000038
B_P000001
B_P000002
B_P000003
B_P000004
B_P000005
B_P000007
B_P000009
B_P000010
B_P000011
B_P000012
B_P000013
B_P000014
B_P000015
B_P000016
B_P000017
B_P000019
B_P000020
C_P000021
C_P000022
C_P000023
C_P000024
C_P000025
C_P000026
C_P000027
C_P000028
C_P000029
C_P000030
C_P000031
C_P000032
C_P000033
C_P000034
C_P000035
C_P000036
C_P000037
C_P000038
C_P000039
C_P000040
D_P000001
D_P000002
D_P000003
D_P000004
D_P000005
D_P000006
D_P000007
D_P000009
D_P000010
D_P000011
D_P000012
D_P000013
D_P000014
D_P000015
D_P000016
D_P000017
D_P000018
D_P000019


In [12]:
wsa_dir = f"{dataset_annotation_path}" + r"/*_polygon.xml"
print(wsa_dir)
wsa_list = glob.glob(wsa_dir)
print(wsa_list)

../data/monkey-data/annotations_polygon/*_polygon.xml
['../data/monkey-data/annotations_polygon/B_P000004_polygon.xml', '../data/monkey-data/annotations_polygon/D_P000011_polygon.xml', '../data/monkey-data/annotations_polygon/B_P000005_polygon.xml', '../data/monkey-data/annotations_polygon/A_P000036_polygon.xml', '../data/monkey-data/annotations_polygon/A_P000001_polygon.xml', '../data/monkey-data/annotations_polygon/D_P000004_polygon.xml', '../data/monkey-data/annotations_polygon/A_P000011_polygon.xml', '../data/monkey-data/annotations_polygon/C_P000031_polygon.xml', '../data/monkey-data/annotations_polygon/A_P000029_polygon.xml', '../data/monkey-data/annotations_polygon/D_P000010_polygon.xml', '../data/monkey-data/annotations_polygon/C_P000033_polygon.xml', '../data/monkey-data/annotations_polygon/A_P000035_polygon.xml', '../data/monkey-data/annotations_polygon/D_P000015_polygon.xml', '../data/monkey-data/annotations_polygon/C_P000036_polygon.xml', '../data/monkey-data/annotations_po

In [ ]:
dataset_path = dataset_config["path"]
dataset_annotation_path = os.path.join(
    dataset_path, dataset_config["annotation_polygon_dir"]
)


def create_dataset_yaml_dict(
    dataset_path: str, annotations_path: str, output_dir: str, output_name: str
):
    # metadata annotation path
    metadata_file = os.path.join(dataset_path, "metadata", "context-information.xlsx")

    # import and pre-process metadata dataframe
    metadata_df = pd.read_excel(metadata_file, index_col=0)
    # remove index
    metadata_df.reset_index(drop=True, inplace=True)
    # replace "x" with "Not available" in the dataframe
    metadata_df.replace("x", "Not available", inplace=True)

    wsa_dir = f"{annotations_path}" + r"/*_polygon.xml"
    wsi_pas_cpg_dir = os.path.join(dataset_path, "images", "pas-cpg")
    wsi_ihc_dir = os.path.join(dataset_path, "images", "ihc")

    wsa_list = glob.glob(wsa_dir)

    # yaml_dict = {"training": []}
    yaml_dict = {"training": [], "validation": []}

    for wsa in wsa_list:
        patient_name = os.path.basename(wsa).split(
            os.path.basename(wsa_dir).split("*")[1]
        )[0]
        print(patient_name)

        # check if the WSI file exists for the PAS_CPG images
        if os.path.isfile(os.path.join(wsi_pas_cpg_dir, patient_name + "_PAS_CPG.tif")):
            print("match found:    ", patient_name)
            wsi = os.path.join(wsi_pas_cpg_dir, patient_name + "_PAS_CPG.tif")
            # add the path to the dataframe df
            metadata_df[metadata_df["Slide ID"] == patient_name]["WSI PAS_CPG Path"] = (
                wsi
            )

            # check if IHC wsi is present for the patient
            if os.path.isfile(os.path.join(wsi_ihc_dir, patient_name + "_IHC_CPG.tif")):
                wsi = os.path.join(wsi_dir, patient_name + "_IHC_CPG.tif")
                metadata_df[metadata_df["Slide ID"] == patient_name][
                    "WSI IHC_CPG Path"
                ] = wsi

            # check if PAS-Diagnostic wsi is present for the patient
            if os.path.isfile(
                os.path.join(wsi_ihc_dir, patient_name + "_PAS_Diagnostic.tif")
            ):
                wsi = os.path.join(wsi_dir, patient_name + "_PAS_Diagnostic.tif")
                metadata_df[metadata_df["Slide ID"] == patient_name][
                    "WSI PAS_Diagnostic Path"
                ] = wsi

            # check if PAS-Original wsi is present for the patient
            if os.path.isfile(
                os.path.join(wsi_ihc_dir, patient_name + "_PAS_Original.tif")
            ):
                wsi = os.path.join(wsi_dir, patient_name + "_PAS_Original.tif")
                metadata_df[metadata_df["Slide ID"] == patient_name][
                    "WSI PAS_Original Path"
                ] = wsi
        else:
            print("no match found:    ", patient_name)

            # yaml_dict["training"].append(
            #     {"wsa": {"path": str(wsa)}, "wsi": {"path": str(wsi)}}
            # )

            # # validation if needed
            # yaml_dict['validation'].append(
            #         {"wsa": {"path": str(wsa)}, "wsi": {"path": str(wsi)}})


wsi_dir = r"./data/monkey-data/images/pas-cpg"  # r'./data_debug/images/pas-cpg'
wsa_dir = r"./data/monkey-data/annotations_polygon/*_polygon.xml"
output_dir = r"./source/configs"
output_name = "training_full.yml"

folders_to_yml(wsi_dir, wsa_dir, output_dir, output_name)